In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model, model_from_json
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import he_normal
import scipy.misc
from matplotlib.pyplot import imshow, imread
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

import sys
sys.path.append("../code")
from residual_blocks import identity_block_small, identity_block_large, convolutional_block_small, convolutional_block_large

## Build the small or large models
Run either build_model_small for model with small building block or build_model_large for model with large building blocks,

In [ ]:
input_shape = (96, 96, 3)
classes = 1

def build_model_small(hp):
    """
    Implementation of a ResNet model with varying amount of stages and id blocks using small residual blocks

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=he_normal(seed=None))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block_small(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block_small(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block_small(X, 3, [64, 64, 256], stage=2, block='c')

    #Y = 20 #for 1 additional stage, 
    #Y = 18 #for 2 additional stages 
    #Y = 16 #for 3 additional stages
    
    # Stage 3 = first additional stage
    X = convolutional_block_small(X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
    for i in range(hp.Int('n_IDblocks1', 1, Y)): 
        X = identity_block_small(X, 3, [128,128,512], stage=3, block=f'_{i}_')

    # Stage 4 = second additional stage (optional, comment when not used)
    X = convolutional_block_small(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
    for i in range(hp.Int('n_IDblocks2', 1, Y)):
        X = identity_block_small(X, 3, [256,256,1024], stage=4, block=f'_{i}_')
    
    # Stage 5 = third additional stage (optional, comment when not used)
    X = convolutional_block_small(X, f=3, filters=[512,512,2084], stage=5, block='a', s=2)
    for i in range(hp.Int('n_IDblocks3', 1, Y)):
        X = identity_block_small(X, 3, [512,512,2084], stage=5, block=f'_{i}_')
    
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)" (comment when using stage 5)
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = he_normal(seed=None))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)
    
    model.compile(SGD(lr=1e-2, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
input_shape = (96, 96, 3)
classes = 1

def build_model_large(hp):
    """
    Implementation of a ResNet model with varying amount of stages and id blocks using large residual blocks

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=he_normal(seed=None))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block_large(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block_large(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block_large(X, 3, [64, 64, 256], stage=2, block='c')

    Y = 62 #for 1 additional stage, 
    #Y = 31 #for 2 additional stages 
    #Y = 20 #for 3 additional stages
    
    # Stage 3 = first additional stage
    X = convolutional_block_large(X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
    for i in range(hp.Int('n_IDblocks1', 1, Y)): 
        X = identity_block_large(X, 3, [128,128,512], stage=3, block=f'_{i}_')

    # Stage 4 = second additional stage (optional, comment when not used)
    X = convolutional_block_large(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
    for i in range(hp.Int('n_IDblocks2', 1, Y)):
        X = identity_block_large(X, 3, [256,256,1024], stage=4, block=f'_{i}_')
    
    # Stage 5 = third additional stage (optional, comment when not used)
    X = convolutional_block_large(X, f=3, filters=[512,512,2084], stage=5, block='a', s=2)
    for i in range(hp.Int('n_IDblocks3', 1, Y)):
        X = identity_block_large(X, 3, [512,512,2084], stage=5, block=f'_{i}_')
    
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)" (comment when using stage 5)
    #X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = he_normal(seed=None))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)
    
    model.compile(SGD(lr=1e-2, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

## Import dataset

In [ ]:
########### insert hyperparameters ################
train_batch_size = 32
val_batch_size = 32
###################################################

# Change these variables to point at the locations and names of the test dataset and your models!!
base_dir = r'C:\Users\20153761\Documents\TUe\4e jaar\3e kwartiel\BIA\train+val'
test_dir = r'C:\Users\20153761\Documents\TUe\4e jaar\3e kwartiel\BIA\test\test'

# dataset parameters
TRAIN_PATH = os.path.join(base_dir, 'train')
VALID_PATH = os.path.join(base_dir, 'valid')
RESCALING_FACTOR = 1./255
IMAGE_SIZE = 96

# instantiate data generators
datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

train_gen = datagen.flow_from_directory(TRAIN_PATH,
                                    target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                    batch_size=train_batch_size,
                                    class_mode='binary',
                                    shuffle=True)

val_gen = datagen.flow_from_directory(VALID_PATH,
                                    target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                    batch_size=val_batch_size,
                                    class_mode='binary',
                                    shuffle=False)

# form steps
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

## KerasTuner for hyperparameter optimalisation using hyperband 

In [ ]:
# directory and project_name should be changed for different models
tuner = Hyperband(
    build_model_large, # choose between build_model_small or build_model_large
    overwrite=True,
    objective='val_accuracy',
    max_epochs = 15,
    factor = 2,
    directory ='storage/ResnetTuner_L',
    project_name='ResNet_Stage12345_L'
)

# stop early to shorten tuning duration
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# search for optimal hyperparameters
tuner.search(train_gen,
             verbose=1, 
             epochs=25,
             batch_size=32,
             steps_per_epoch=train_steps,
             callbacks=[stop_early],
             validation_steps=val_steps,
             validation_data=val_gen)

# get best hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

## Fit best model

In [ ]:
# save the model and weights (change names!)
model = tuner.hypermodel.build(best_hps)
model_name = 'ResNet_Stage12345_L'
model_filepath = './' + 'storage' + '/' + model_name + '.json'
weights_filepath = './' + 'storage' + '/' + model_name + '_weights.hdf5'

# serialize model to JSON
model_json = model.to_json()
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json) 
    
# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]

# fit model
history = model.fit(train_gen, 
          epochs = 25, 
          batch_size = 32, 
          steps_per_epoch=train_steps,
          validation_data=val_gen,
          validation_steps=val_steps,
          callbacks=callbacks_list)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
# summarize model
model.summary()

## Save submission.csv for kaggle

In [ ]:
# load model and model weights
json_file = open(model_filepath, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)


# load weights into new model
model.load_weights(weights_filepath)


# open the test set in batches (as it is a very big dataset) and make predictions
test_files = glob.glob(test_dir + '*')
submission = pd.DataFrame()

file_batch = 5000
max_idx = len(test_files)

for idx in range(0, max_idx, file_batch):

    print('Indexes: %i - %i'%(idx, idx+file_batch))

    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})


    # get the image id 
    test_df['id'] = test_df.path.map(lambda x: x.split(os.sep)[-1].split('.')[0])
    test_df['image'] = test_df['path'].map(imread)
    
    
    K_test = np.stack(test_df['image'].values)
    
    # apply the same preprocessing as during draining
    K_test = K_test.astype('float')/255.0
    
    predictions = model.predict(K_test)
    
    test_df['label'] = predictions
    submission = pd.concat([submission, test_df[['id', 'label']]])


# save your submission (change name!)
submission.head()
submission.to_csv('submission_ResNet_Stage12345_L.csv', index = False, header = True)

Start TensorBoard by calling the following commands from the Anaconda Prompt command line:

````bash
activate 8p361
cd 'path/where/logs/are'
tensorboard --logdir logs
````

Navigate browser to http://localhost:6006/ to visualize the training and validation loss curves.